In [14]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib
from matplotlib.axes import Axes
%matplotlib inline

import numpy as np
import pyvista as pv
import pandas as pd
import xarray as xr
import time

import vtk
from vtk import *

import pyvista as pv
from pyvista import examples
pv.global_theme.jupyter_backend = 'panel'

import cartopy.crs as ccrs

In [3]:
pv.global_theme.jupyter_backend = 'pythreejs'

mesh = examples.download_bunny()
mesh.flip_normals()

pl = pv.Plotter()
pl.add_mesh(mesh, color='lightgrey')
pl.camera_position = 'xy'
pl.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [13]:
print(mesh)

PolyData (0x7f4ffe086520)
  N Cells:	69451
  N Points:	35947
  X Bounds:	-9.469e-02, 6.101e-02
  Y Bounds:	3.299e-02, 1.873e-01
  Z Bounds:	-6.187e-02, 5.880e-02
  N Arrays:	0



In [4]:
ds = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_31/output_WAOM_check/ocean_flt.nc')
#dt = xr.open_dataset('D:/run_5_31/ocean_avg_0001.nc')
#dd = xr.open_dataset('/scratch/project_2000789/muramarg/run_5_31/output_WAOM_check/ocean_avg_0001.nc')
#dg = xr.open_dataset('/scratch/project_2000789/boeiradi/waom10_frc/waom10extend_grd.nc')

In [5]:
x = (ds.variables['Xgrid'].values)
y = (ds.variables['Ygrid'].values)
z = (ds.variables['Zgrid'].values)

In [6]:
print(x.shape[0])

23521


In [7]:
def make_points(i):
    """Helper to make XYZ points"""
    z1 = list(z[:,i])
    x1 = list(x[:,i])
    y1 = list(y[:,i])
    return np.column_stack((x1, y1, z1))

In [8]:
points = make_points(5)

In [9]:
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly


line = lines_from_points(points)
line

PolyData,Information
N Cells,23520
N Points,23521
X Bounds,"5.127e+02, 5.450e+02"
Y Bounds,"3.390e+02, 3.788e+02"
Z Bounds,"2.445e+01, 3.079e+01"
N Arrays,0


In [10]:
pv.global_theme.jupyter_backend = 'pythreejs'

In [11]:
def cast_all_points():
    #pv.start_xvfb()
    start_time = time.time()
    network = pv.MultiBlock()
    network.add_field_data(np.arange(x.shape[0]/900),"time")
    
    # add backend
    
    for i in range(0,3):
        if (i%5 ==0):
            print(i,'/ 105','in',(time.time()-start_time)/60)
        points = make_points(i)
        line = lines_from_points(points)
        line["days since start"] = np.arange(line.n_points)
        tube = line.tube(radius=0.1)
        network.append(tube)
        #line['time'] = np.arange(x.shape[0])
        
        #tube.plot(smooth_shading=True)
        
    # create the scalar bars and plot
    print('ok')
    
    p = pv.Plotter()
    sargs = dict(
        n_labels=0,
        label_font_size = 14
    )
    annotations = {
        0:'0',2820: '30', 5640: '60', 8460:'90',11280: '120', 14100:'150', 16920:'180',
        19740:'210', 22560:'240',   
    }
    mesh
    #_= p.add_mesh(network, smooth_shading=True,annotations=annotations, scalar_bar_args=sargs)
    
    # add the zice shape
    print('plotting')
    # add the plot axes
    _ = p.add_axes(line_width=5, labels_off=False)
    p.show()
    #p.save_graphic("testing.svg") 
    print('total time:',(time.time()-start_time)/60,'min')
cast_all_points()

0 / 105 in 4.792213439941407e-06
ok
plotting


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

total time: 0.053670438130696614 min


In [ ]:
def cast_all_points():
    renderer = vtk.vtkRenderer()
    renderer.SetBackground(1.0, 1.0, 1.0)
    
    start_time = time.time()
    network = pv.MultiBlock()
    network.add_field_data(np.arange(x.shape[0]/900),"time")
    #print(network)
    for i in range(0,10):
        if (i%5 ==0):
            print(i,'/ 105','in',(time.time()-start_time)/60)
        points = make_points(i)
        line = lines_from_points(points)
        line["days since start"] = np.arange(line.n_points)
        tube = line.tube(radius=0.1)
        network.append(tube)
        #line['time'] = np.arange(x.shape[0])
        
        #tube.plot(smooth_shading=True)
    renderer.AddActor(network)
    
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)
    
#     # create the scalar bars and plot
#     p = pv.Plotter()
#     sargs = dict(
#         n_labels=0,
#         label_font_size = 14
#     )
#     annotations = {
#         0:'0',2820: '30', 5640: '60', 8460:'90',11280: '120', 14100:'150', 16920:'180',
#         19740:'210', 22560:'240',   
#     }
#     p.add_mesh(network, smooth_shading=True,annotations=annotations, scalar_bar_args=sargs)
    
#     # add the zice shape
    
#     # add the plot axes
#     _ = p.add_axes(line_width=5, labels_off=False)
#     p.save_graphic("img.svg") 
    print('total time:',(time.time()-start_time)/60,'min')
cast_all_points()

In [ ]:
dt.variables['zice'].values

In [ ]:
# create path from zice
for row in dt.variables['zice'].values:
    #print(row)
    for i in row:
        #print(i)
        if i != 0:
            print(i)

In [ ]:
# def make_points(i):
#     """Helper to make XYZ points"""
#     z1 = list(z[:,i])
#     x1 = list(x[:,i])
#     y1 = list(y[:,i])
#     return np.column_stack((x1, y1, z1))
# points = make_points(1)
# line = lines_from_points(points)

In [ ]:
# line["scalars"] = np.arange(line.n_points)
# tube = line.tube(radius=0.1)
# tube.plot(smooth_shading=True)

In [ ]:
# p = pv.Plotter()
# p.add_mesh(stream.tube(radius=0.0015))
# p.view_xy()
# p.show(cpos=cpos)